<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/%EC%9E%90%EB%8F%99%EC%B0%A8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 모든 댓글 크롤링


In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import json
import re

## 1차 차종리스트 및 1차 URL 만들고 저장하기

In [ ]:
# 네이버에 '자동차' 검색, html이랑 java script랑 혼용 
# 국내시판 끄고 전체 자동차 가져오기, json 형태
url='https://m.search.naver.com/p/csearch/content/nqapirender.nhn?pkid=128&where=nexearch&key=car_list&u1=0&u3=&u7=&u5=0~7&u6=&u4=&u8=&u9=&u2=%EC%B6%9C%EC%8B%9C%EC%88%9C%7C'
resp=requests.get(url)
jsn=json.loads(resp.content)

# 토탈 페이지수 가져오기, 한 페이지에 기본 차량 8대
total_page=int(jsn['total']) # 총 페이지수 20221105 기준 총 1236 페이지

In [ ]:
# 모든 차량 링크 가져오기
# 방안 1
# json의 'html'라는 키의 밸류가 html형태의 한개의 문자열에 다 적혀있음

first_car_list=[] # 1차 차종리스트
first_url_list=[] # 1치 링크리스트

# 이건 나중에 고려
# first_price_list=[] # 1차 금액리스트
# first_day_list=[] # 1차 출시일 리스트

for page in tqdm(range(total_page)):

    url='https://m.search.naver.com/p/csearch/content/nqapirender.nhn?pkid=128&where=nexearch&key=car_list&u1={}&u3=&u7=&u5=0~7&u6=&u4=&u8=&u9=&u2=%EC%B6%9C%EC%8B%9C%EC%88%9C%7C'.format(page)
    resp=requests.get(url)
    jsn=json.loads(resp.content)['html']
    soup=BeautifulSoup(jsn, 'lxml')
    info_tags=soup.select('strong.title a')

    for info in info_tags:
        first_car_list.append(info.text) # 차이름 추가
        first_url_list.append('https://search.naver.com/search.naver'+info['href']) # 차링크 추가


In [ ]:
# csv로 만들어서 저장
first_df=pd.DataFrame([first_car_list, first_url_list]).transpose() # 행과 열을 뒤집어야 원하는 형태로 나옴
first_df.columns=['first_car_list','first_url_list'] # 컬럼이름 설정
first_df.to_csv('/content/drive/MyDrive/프로젝트_1/first_DataFrame.csv', encoding='utf-8-sig')

## 2차 차종리스트 및 URL 만들고 저장하기

In [ ]:
# csv 파일 불러옴
first_df=pd.read_csv('/content/drive/MyDrive/1차_차종_데이터/first_DataFrame.csv', index_col=0)

In [ ]:
second_car_list=[] # 2차 차종 리스트
second_url_list=[] # 2차 링크 오너평가 리스트
for i in tqdm(range(len(first_df))):

    url=first_df['first_url_list'][i] # 1차 차종리스트에서 차종선택
    resp=requests.get(url)
    soup_category=BeautifulSoup(resp.content, 'lxml')
    a_tags=soup_category.select('li.tab._tab a') # 탭 항목들 가져옴(전체, 기본정보, 제원, 오너평가, 포토 등)

    if a_tags[3].text.strip() == '오너평가': # 댓글이 있는 차종만 가져오기 위한 '오너평가'
        second_car_list.append(first_df['first_car_list'][i]) # 2차 차종리스트
        second_url_list.append('https://search.naver.com/search.naver'+a_tags[3]['href']) # 2차 링크 오너평가 리스트
print(len(second_car_list), len(second_url_list))

  0%|          | 0/9883 [00:00<?, ?it/s]

4269 4269


In [ ]:
# csv로 만들어서 저장
second_df=pd.DataFrame([second_car_list, second_url_list]).transpose() # 행과 열을 뒤집어야 원하는 형태로 나옴
second_df.columns=['second_car_list','second_url_list'] # 컬럼이름 설정
second_df.to_csv('/content/drive/MyDrive/2차_차종_데이터/second_DataFrame.csv', encoding='utf-8-sig')

## 2차 차종 댓글 가져오기

In [ ]:
# 2차 csv 파일 불러옴
second_df=pd.read_csv('/content/drive/MyDrive/2차_차종_데이터/second_DataFrame.csv', index_col=0)

In [ ]:
total_comment_list=[] # 댓글 리스트
img_list=[] # 이미지 리스트
comment_count=0 # 댓글 카운트용
# 댓글 가져오기
for i in tqdm(range(len(second_df))):
    url=second_df['second_url_list'][i]
    resp=requests.get(url)
    soup=BeautifulSoup(resp.content, 'lxml')
    # groupId (차종)와 objectId (모델) 뽑아내기
    # 모델이 한종류만 있거나 종합의 경우 groupId와 objectId는 동일하다
    groupId=soup.select('ul.tab_list._tab_list li.tab')[0]['data-group'] # 모델에 상관없이 맨처음[0]에 위치한 종합거 가져오면됨
    objectId=soup.select('ul.tab_list._tab_list li.tab')[0]['data-kgs-option'] # 모델명들 가져오는거 모든인덱스값들만 가져오게하면 나중에 활용가능
    os_num=re.findall('os=(.+?)&',second_df['second_url_list'][i])[0] # 자동차 링크에 쓰이는 번호


    url='https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'

    headers={'referer': 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os={}&qvt=0&query=2023%20%ED%8F%AD%EC%8A%A4%EB%B0%94%EA%B2%90%20%EC%A0%9C%ED%83%80%20%EC%98%A4%EB%84%88%ED%8F%89%EA%B0%80'.format(os_num),
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

    params={'ticket':'mycar',
            'templateId':'default',
            'pool':'cbox12',
            '_callback':'',
            'lang':'ko',
            'country':'KR',
            'objectId':groupId, # 모델, 차종의 종합을 가져올거면 objectId가 아니라 groupId 가져오면됨
            'pageSize':10,
            'indexSize':10,
            'groupId':groupId, # 차종
            'listType':'group',
            'pageType':'more',
            'page':1,
            'refresh':'false',
            'sort':'ALL'
            }
    
    resp1=requests.get(url, headers=headers, params=params)
    jsn=json.loads(resp1.content[10:-2])
    commentList=jsn['result']['commentList']
    totalPage=jsn['result']['pageModel']['totalPages'] # 댓글 페이지 총수 가져오기
    comment_list=[]
    for page in range(1,totalPage+1): # 페이지 1번부터 시작
        params['page']=page
        resp1=requests.get(url, headers=headers, params=params)
        result=resp1.text[10:-2]
        jsn=json.loads(result)
        commentList=jsn['result']['commentList']
        for comment in commentList:
            comment_list+=[comment['contents'].strip()] # 대괄호[] 안씌우면 한글자씩 저장됨
    total_comment_list+=[comment_list]
    comment_count+=len(comment_list) # 댓글 카운트용
    # print(second_df['second_car_list'][i],'/ 추가된 댓글 수',len(comment_list),'/ 총 댓글 수:', comment_count) # 진행도 확인용

In [ ]:
# 2차 차종데이터프레임에 댓글들 추가하고 저장하기
second_df['comments']=total_comment_list
second_df.to_csv('/content/drive/MyDrive/2차_차종_데이터/second_comments_DataFrame.csv', encoding='utf-8-sig')

In [ ]:
# 똑같은 댓글들을 가지는 애들이 있음
# 벤츠나 미니 해치백 등 몇몇 하이브리드모델과 기존모델이 이러한 경우들이 있음
# 하나의 평가가 여려군데에 적용되는건 유효한 데이터가 아님
# 집합처리해서 중복제거한 것을 텍스트 전처리해야함

total_comment_set=set([])
for comment in total_comment_list:
    total_comment_set.update(comment)

# 중복제거된 댓글들 저장하기
with open('/content/drive/MyDrive/2차_차종_데이터/total_comment_set','w',encoding='utf-8-sig') as f:
    for i in total_comment_set:
        f.write(i+'\n') # 한줄에 한개의 댓글식 저장

In [ ]:
# 데이터 확인
second_df_reviews=pd.read_csv('/content/drive/MyDrive/프로젝트_1/second_comments_DataFrame.csv', index_col=0)
second_df_reviews

,second_car_list,second_url_list,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한..."
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...
...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"['레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로..."
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,['아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시...
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,['깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해...
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,['쓸만한 차\n가성비 좋음\n연비좋음\n. ...


## 2차 차종의 브랜드, 연도, 가격 추가

In [ ]:
# 2차 csv 파일 불러와서 브랜드, 연도, 가격 추가
second_df=pd.read_csv('/content/drive/MyDrive/프로젝트_1/second_DataFrame.csv', index_col=0)
second_df

,second_car_list,second_url_list
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...
...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...


In [ ]:
second_df.second_url_list[0]

'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os=28917739&qvt=0&query=2023%20%ED%8F%AD%EC%8A%A4%EB%B0%94%EA%B2%90%20%EC%A0%9C%ED%83%80%20%EC%98%A4%EB%84%88%ED%8F%89%EA%B0%80'

In [ ]:
# 현재 오너평가 링크로 저장되어있어서, 기본 전체링크로 변경해주기위해 '%오너평가' 부분 삭제
second_df.second_url_list.replace('%20%EC%98%A4%EB%84%88%ED%8F%89%EA%B0%80', '', inplace=True, regex=True) # 문자열 일부만 치환하고 싶은 경우는 regex=True 해주어야 함

In [ ]:
second_df.second_url_list[0]

'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os=28917739&qvt=0&query=2023%20%ED%8F%AD%EC%8A%A4%EB%B0%94%EA%B2%90%20%EC%A0%9C%ED%83%80'

In [ ]:
from urllib import parse # url 디코딩 모듈

In [ ]:
# 한 페이지 연습

url = second_df.second_url_list[0]
resp=requests.get(url)
soup=BeautifulSoup(resp.content, 'lxml')

price = soup.select('div.info_group dd')[0].text.split('~')[0] # 최소가격
shape = soup.select('div.sub_title span.txt')[0].text
year = soup.select('div.sub_title span.txt')[1].text

q_string = soup.select('div.logo_area a')[0]['href'].split('=')[-1] # 브랜드 링크 쿼리스트링
brand = parse.unquote(q_string) # url 디코딩

In [ ]:
brand_list=[] # 제조사 리스트
price_list=[] # 가격 리스트(최소)
year_list=[] # 출시년도 리스트
shape_list=[] # 외형 리스트

for i in tqdm(range(len(second_df))):
    url=second_df['second_url_list'][i]
    resp=requests.get(url)
    soup=BeautifulSoup(resp.content, 'lxml')

    price = soup.select('div.info_group dd')[0].text.split('~')[0] # 최소가격
    shape = soup.select('div.sub_title span.txt')[0].text
    year = soup.select('div.sub_title span.txt')[1].text

    q_string = soup.select('div.logo_area a')[0]['href'].split('=')[-1] # 브랜드 링크 쿼리스트링
    brand = parse.unquote(q_string) # url 디코딩

    brand_list.append(brand)
    price_list.append(price)
    shape_list.append(shape)
    year_list.append(year)

  0%|          | 0/4269 [00:00<?, ?it/s]

In [ ]:
add_car_info = {
    '제조사' : brand_list,
    '가격' : price_list,
    '출시년도' : year_list,
    '외형' : shape_list
}

df_add = pd.DataFrame(add_car_info)

In [ ]:
df_add # 행 갯수 같은 것 확인

,제조사,가격,출시년도,외형
0,폭스바겐,"3,232",2023,준중형 세단
1,캐딜락,"5,851만원 내차시세",2023,준중형 SUV
2,아우디,"10,997만원 내차시세",2023,준대형 세단
3,아우디,"8,116만원 내차시세",2023,중형 세단
4,아우디,"7,182",2023,중형 SUV
...,...,...,...,...
4264,닛산,가솔린,1991,경형 컨버터블
4265,현대,가솔린,1990,소형 해치백
4266,기아,629만원 내차시세,1990,소형 세단
4267,현대,649,1990,준중형 세단


In [ ]:
type(df_add)

pandas.core.frame.DataFrame

In [ ]:
df_add.to_csv('add_car_info.csv', encoding='utf-8-sig')

In [ ]:
df_add=pd.read_csv('/content/drive/MyDrive/프로젝트_1/add_car_info.csv', index_col=0)
df_add

,제조사,가격,출시년도,외형
0,폭스바겐,"3,232",2023,준중형 세단
1,캐딜락,"5,851만원 내차시세",2023,준중형 SUV
2,아우디,"10,997만원 내차시세",2023,준대형 세단
3,아우디,"8,116만원 내차시세",2023,중형 세단
4,아우디,"7,182",2023,중형 SUV
...,...,...,...,...
4264,닛산,가솔린,1991,경형 컨버터블
4265,현대,가솔린,1990,소형 해치백
4266,기아,629만원 내차시세,1990,소형 세단
4267,현대,649,1990,준중형 세단


In [ ]:
# 크롤링 실수로 잘못된 가격열 삭제
df_add = df_add.drop(['가격'], axis=1)
df_add

,제조사,출시년도,외형
0,폭스바겐,2023,준중형 세단
1,캐딜락,2023,준중형 SUV
2,아우디,2023,준대형 세단
3,아우디,2023,중형 세단
4,아우디,2023,중형 SUV
...,...,...,...
4264,닛산,1991,경형 컨버터블
4265,현대,1990,소형 해치백
4266,기아,1990,소형 세단
4267,현대,1990,준중형 세단


In [ ]:
# 위에서 가져온 리스트를 댓글 추출한 df 와 합치기
# 열의 개수 확인 후 concat 함수 사용해서 합칠 수 있음 
second_df_reviews=pd.read_csv('/content/drive/MyDrive/프로젝트_1/second_comments_DataFrame.csv', index_col=0)
second_df_reviews

,second_car_list,second_url_list,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한..."
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...
...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"['레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로..."
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,['아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시...
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,['깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해...
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,['쓸만한 차\n가성비 좋음\n연비좋음\n. ...


In [ ]:
# comment 가 현재 str형태로, unnest 함수 사용 위해 list로 변환
for i in range(len(second_df_reviews.comments)):
    second_df_reviews.comments[i] = second_df_reviews.comments[i][2:-3].split("', '")  # 맨앞[' 와 맨뒤 '] 제거후 '. '로 split 하면 list로 변경 가능


In [ ]:
# 중첩배열  https://ahnty0122.tistory.com/10 참고 
import numpy as np
import pandas as pd

def unnest(df, tile, explode):
    vals = df[explode].sum(1)
    rs = [len(r) for r in vals]
    a = np.repeat(df[tile].values, rs, axis=0)
    b = np.concatenate(vals.values)
    d = np.column_stack((a, b))
    return pd.DataFrame(d, columns = tile +  ['_'.join(explode)])

In [ ]:
# 우선 2개의 df 합치기
df_total = pd.concat([second_df_reviews, df_add], axis=1)
df_total

,second_car_list,second_url_list,comments,제조사,출시년도,외형
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,[여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자체...,폭스바겐,2023,준중형 세단
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"[23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한 ...",캐딜락,2023,준중형 SUV
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,[제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 편...,아우디,2023,준대형 세단
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,[승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만족...,아우디,2023,중형 세단
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,[볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 타...,아우디,2023,중형 SUV
...,...,...,...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"[레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로망...",닛산,1991,경형 컨버터블
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,[아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시는...,현대,1990,소형 해치백
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,[깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해야...,기아,1990,소형 세단
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,[쓸만한 차\n가성비 좋음\n연비좋음\n. ....,현대,1990,준중형 세단


In [ ]:
df_total = unnest(df_total, ['second_car_list','second_url_list','제조사','출시년도','외형'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_total

# comment 늘이기


## 1000행 늘이기 되는 것 확인


In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import json
import re
import numpy as np

In [2]:
# 단순히 늘이기 먼저 시도
second_df_reviews=pd.read_csv('/content/drive/MyDrive/프로젝트_1/second_comments_DataFrame.csv', index_col=0)
second_df_reviews

,second_car_list,second_url_list,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한..."
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...
...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"['레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로..."
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,['아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시...
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,['깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해...
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,['쓸만한 차\n가성비 좋음\n연비좋음\n. ...


In [3]:
# comment 가 현재 str형태로, unnest 함수 사용 위해 list로 변환
for i in range(len(second_df_reviews.comments)):
    second_df_reviews.comments[i] = second_df_reviews.comments[i][2:-3].split("', '")  # 맨앞[' 와 맨뒤 '] 제거후 '. '로 split 하면 list로 변경 가능

In [4]:
# 중첩배열  https://ahnty0122.tistory.com/10 참고 
def unnest(df, tile, explode):
    vals = df[explode].sum(1)
    rs = [len(r) for r in vals]
    a = np.repeat(df[tile].values, rs, axis=0)
    b = np.concatenate(vals.values)
    d = np.column_stack((a, b))
    return pd.DataFrame(d, columns = tile +  ['_'.join(explode)])

In [ ]:
# df_reviews = unnest(second_df_reviews, ['second_car_list','second_url_list'], ['comments'])
df_reviews = unnest(second_df_reviews, ['second_car_list'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_reviews

In [3]:
# 1000행만 시도 
df_test = second_df_reviews.loc[:1000]
df_test

,second_car_list,second_url_list,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한..."
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...
...,...,...,...
996,2019 렉스턴 스포츠 칸,https://search.naver.com/search.naver?where=ne...,"['캠핑용 차량으로 최고입니다 굿굿굿굿 굿굿굿 \n픽업트럭 대체불가 차량입니다', ..."
997,2019 렉스턴 스포츠,https://search.naver.com/search.naver?where=ne...,['정말좋다? 튼튼하고 안정적이며 디자인도 참좋다! 가격대비 100% 최고...
998,2019 렉서스 LS 하이브리드,https://search.naver.com/search.naver?where=ne...,['생에 마지막 차량으로 구입했습니다.\n차량 운행 만족한 편입니다. 하이브리드 특...
999,2019 렉서스 LS,https://search.naver.com/search.naver?where=ne...,['생에 마지막 차량으로 구입했습니다.\n차량 운행 만족한 편입니다. 하이브리드 특...


In [4]:
# comment 가 현재 str형태로, unnest 함수 사용 위해 list로 변환
for i in range(len(df_test.comments)):
    df_test.comments[i] = df_test.comments[i][2:-3].split("', '")  # 맨앞[' 와 맨뒤 '] 제거후 '. '로 split 하면 list로 변경 가능

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# 중첩배열  https://ahnty0122.tistory.com/10 참고 
def unnest(df, tile, explode):
    vals = df[explode].sum(1)
    rs = [len(r) for r in vals]
    a = np.repeat(df[tile].values, rs, axis=0)
    b = np.concatenate(vals.values)
    d = np.column_stack((a, b))
    return pd.DataFrame(d, columns = tile +  ['_'.join(explode)])

In [6]:
type(df_test.comments[0])

list

In [7]:
# df_reviews = unnest(second_df_reviews, ['second_car_list','second_url_list'], ['comments'])
df_test = unnest(df_test, ['second_car_list'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_test

,second_car_list,comments
0,2023 폭스바겐 제타,여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자체 ...
1,2023 폭스바겐 제타,저는 차알못이고 신랑이 타는데\n연비는 진짜 미쳤구요\n이전에 미니 타다가 변경해서...
2,2023 폭스바겐 제타,가격이 저렴하고 무난한 디자인. 기본 옵션들도 부족함이 별로 없다 생각합니다. 다만...
3,2023 폭스바겐 제타,다 좋음 연비좋고 다 좋은데 서비스센터 구리고 엔진노킹 바퀴 소음 디스크문제 해결안...
4,2023 폭스바겐 제타,시내주행은 연비 그닥이고 고속도로 정속주행시 연비 20키로 디자인은 개인적으로 깔끔...
...,...,...
354995,2019 렉서스 IS F 스포츠,"좋은차입니다.\n연비도, 와관도, 고장없고..!\n차 구입 후 지금까지 한번의 고장없음."
354996,2019 렉서스 IS F 스포츠,디자인 좋고 내구성은 지금까지 만족합니다 실내 디자인이 올드하고 편의장비가 좀 부족해요
354997,2019 렉서스 IS F 스포츠,대중적이지 않습니다. 한국사람들이 선호하는 독일차 브랜드들에서 최고가 모델을 제외한...
354998,2019 렉서스 IS F 스포츠,8년째 타고 있는 차량입니다.디자인 이쁘고 잔고장이 별로 없습니다.지금은 단종된걸로...


## 1000단위로 행 나눠서 진행


In [14]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import json
import re
import numpy as np

In [15]:
# 기존 댓글 df
second_df_reviews=pd.read_csv('/content/drive/MyDrive/프로젝트_1/second_comments_DataFrame.csv', index_col=0)
second_df_reviews

,second_car_list,second_url_list,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한..."
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...
...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"['레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로..."
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,['아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시...
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,['깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해...
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,['쓸만한 차\n가성비 좋음\n연비좋음\n. ...


In [16]:
# 추가 정보
df_add=pd.read_csv('/content/drive/MyDrive/프로젝트_1/add_car_info.csv', index_col=0)
df_add
# 크롤링 실수로 잘못된 가격열 삭제
df_add = df_add.drop(['가격'], axis=1)
df_add

,제조사,출시년도,외형
0,폭스바겐,2023,준중형 세단
1,캐딜락,2023,준중형 SUV
2,아우디,2023,준대형 세단
3,아우디,2023,중형 세단
4,아우디,2023,중형 SUV
...,...,...,...
4264,닛산,1991,경형 컨버터블
4265,현대,1990,소형 해치백
4266,기아,1990,소형 세단
4267,현대,1990,준중형 세단


In [17]:
# 우선 2개의 df 합치기
df_total = pd.concat([second_df_reviews, df_add], axis=1)
df_total

,second_car_list,second_url_list,comments,제조사,출시년도,외형
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,['여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자...,폭스바겐,2023,준중형 세단
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"['23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한...",캐딜락,2023,준중형 SUV
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,['제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 ...,아우디,2023,준대형 세단
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,['승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만...,아우디,2023,중형 세단
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,['볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 ...,아우디,2023,중형 SUV
...,...,...,...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"['레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로...",닛산,1991,경형 컨버터블
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,['아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시...,현대,1990,소형 해치백
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,['깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해...,기아,1990,소형 세단
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,['쓸만한 차\n가성비 좋음\n연비좋음\n. ...,현대,1990,준중형 세단


In [18]:
# comment 가 현재 str형태로, unnest 함수 사용 위해 list로 변환
for i in range(len(df_total.comments)):
    df_total.comments[i] = df_total.comments[i][2:-3].split("', '")  # 맨앞[' 와 맨뒤 '] 제거후 '. '로 split 하면 list로 변경 가능
df_total

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,second_car_list,second_url_list,comments,제조사,출시년도,외형
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,[여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자체...,폭스바겐,2023,준중형 세단
1,2023 캐딜락 XT4,https://search.naver.com/search.naver?where=ne...,"[23년형으로 받았습니다. SUV답지않은 승차감, 부드러우면서도 안정감있고 단단한 ...",캐딜락,2023,준중형 SUV
2,2023 아우디 S6,https://search.naver.com/search.naver?where=ne...,[제 인생에 있어서 마지막 디젤 세단이 될 것 같네요. 토크빨 맘에 들고 승차감 편...,아우디,2023,준대형 세단
3,2023 아우디 S4,https://search.naver.com/search.naver?where=ne...,[승차감이 생각보다 편해서 놀랍습니다. \n연비는 별로 좋지 않고 기타 부분은 만족...,아우디,2023,중형 세단
4,2023 아우디 Q5 스포트백,https://search.naver.com/search.naver?where=ne...,[볼보 suv 에서 갈아 탔는데 정숙감도 좋고 코너링도 만족해요\n별탈없이 오래 타...,아우디,2023,중형 SUV
...,...,...,...,...,...,...
4264,1991 닛산 휘가로,https://search.naver.com/search.naver?where=ne...,"[레트로카였지만 이제 클래식카가 된, 다시는 발매될리 없는 리미티드 완소 나의 로망...",닛산,1991,경형 컨버터블
4265,1990 엑셀 해치백,https://search.naver.com/search.naver?where=ne...,[아! 2등이네\n엑셀 휄 커버 4개 )구멍4개짜리) 구입하고싶어요..\n아시는...,현대,1990,소형 해치백
4266,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,[깔 수 없는 명차 중의 명차\n기아의 자존심과 자부심\n우리나라에서 다시 출시해야...,기아,1990,소형 세단
4267,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,[쓸만한 차\n가성비 좋음\n연비좋음\n. ....,현대,1990,준중형 세단


In [19]:
# 중첩배열  https://ahnty0122.tistory.com/10 참고 
def unnest(df, tile, explode):
    vals = df[explode].sum(1)
    rs = [len(r) for r in vals]
    a = np.repeat(df[tile].values, rs, axis=0)
    b = np.concatenate(vals.values)
    d = np.column_stack((a, b))
    return pd.DataFrame(d, columns = tile +  ['_'.join(explode)])

In [20]:
# 램 과부화 방지위해 1000행씩 나눠서 진행
df_total_1 = df_total.loc[:1000]
df_total_2 = df_total.loc[1000:2000]
df_total_3 = df_total.loc[2000:3000]
df_total_4 = df_total.loc[3000:]

In [21]:
df_total_1 = unnest(df_total_1, ['second_car_list','second_url_list','제조사','출시년도','외형'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_total_1

,second_car_list,second_url_list,제조사,출시년도,외형,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자체 ...
1,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,저는 차알못이고 신랑이 타는데\n연비는 진짜 미쳤구요\n이전에 미니 타다가 변경해서...
2,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,가격이 저렴하고 무난한 디자인. 기본 옵션들도 부족함이 별로 없다 생각합니다. 다만...
3,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,다 좋음 연비좋고 다 좋은데 서비스센터 구리고 엔진노킹 바퀴 소음 디스크문제 해결안...
4,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,시내주행은 연비 그닥이고 고속도로 정속주행시 연비 20키로 디자인은 개인적으로 깔끔...
...,...,...,...,...,...,...
354995,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,"좋은차입니다.\n연비도, 와관도, 고장없고..!\n차 구입 후 지금까지 한번의 고장없음."
354996,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,디자인 좋고 내구성은 지금까지 만족합니다 실내 디자인이 올드하고 편의장비가 좀 부족해요
354997,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,대중적이지 않습니다. 한국사람들이 선호하는 독일차 브랜드들에서 최고가 모델을 제외한...
354998,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,8년째 타고 있는 차량입니다.디자인 이쁘고 잔고장이 별로 없습니다.지금은 단종된걸로...


In [22]:
df_total_2 = unnest(df_total_2, ['second_car_list','second_url_list','제조사','출시년도','외형'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_total_2

,second_car_list,second_url_list,제조사,출시년도,외형,comments
0,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,차를 구매하기전 꼭 필요한게 무엇인가가 중요합니다.\n가격으로 구매해서 후회한적이 ...
1,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,"정숙하고 승차감 대단히 우수하고 만듦새와 실내외 디자인, 편의장치 모두 우수함. 독..."
2,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,"역시나 조용하고 잔고장 없고, 7년동안 한번도 잔고장 난적이 없고 딱히 뛰어나지도 ..."
3,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,20대에 사서 바꾸고 싶은데 고장이 안나 제발 좀 바꾸게 적당한 고장 좀 나자..잔...
4,2019 렉서스 IS F 스포츠,https://search.naver.com/search.naver?where=ne...,렉서스,2019,중형 세단,"6기통 후륜세단., 차량크기도 적당하고 고급스러우며 무엇보다\n흔하지 않아서 좋아요"
...,...,...,...,...,...,...
349321,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,패션카로는 1등이 아닌가..\n가성비 최고입니당ㅇㅇㅇㅇㅇㄴㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ
349322,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,* 조향시에 또 민첩하게 움직이지만 무언가 헐렁한 그 느낌을 지울 수 없다. 투어러...
349323,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,가성비 좋은 8기통\n고속주행연비는 기대해볼만하나 시내에선 답도없음\n6.5세대부터...
349324,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,"기대이상으로 만족하면서 타고있습니다 승차감,성능 좋구요\n저렴하다고 생각해서 사려고..."


In [23]:
df_total_3 = unnest(df_total_3, ['second_car_list','second_url_list','제조사','출시년도','외형'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_total_3

,second_car_list,second_url_list,제조사,출시년도,외형,comments
0,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,"연비가 나쁘지도 않은거 같애요 생각보다 앞이랑 사이드가 큽니다 조심성이 필요하고,..."
1,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,6세대 5.0 GT 컨버터블 6년간 타보면서 느낌 점 :\n\n 마력대비 가성비는 ...
2,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,감성과 가성비로 타는 스포츠카\n연비주행하면 연비도 나쁘지 않으나 그러기엔 차가 너...
3,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,머슬카 3대장도 역사의 뒤안길로 사라져가고 있습니다. \n쉐보레 카마로의 단종과 닷...
4,2015 포드 머스탱,https://search.naver.com/search.naver?where=ne...,포드,2015,스포츠카 쿠페,연비주행을 위한 차는 아니지만 매우 아쉬움\n이 가격에 이 디자인이면 가성비 극강임...
...,...,...,...,...,...,...
274757,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,차성능과 스마트카로서의 만족하고있으나\n연비는 최신경쟁차종에비해 현저히 떨어진다.
274758,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,8년간 운행해보니 그래도 안전한 차인건 인정. \n고질적으로 고장나는 부분이 있고 ...
274759,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,고속 주행시 안정감 아주좋고 매우 튼튼해서 안전성도 좋습니다. 그러나 트랜스미션의 ...
274760,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,2012년식 알페온! 10년을 탔지만 만족한다. \n10년전에 다른 차를 샀으면 ...


In [24]:
df_total_4 = unnest(df_total_4, ['second_car_list','second_url_list','제조사','출시년도','외형'], ['comments'])
# 기준이 되는 열을 2번째 인자로, 분리할 열을 3번째 인자로 호출
df_total_4

,second_car_list,second_url_list,제조사,출시년도,외형,comments
0,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,뷰익 라크로스를 너무 심하게 다운사이징해서 아쉽긴 했지만\n(그랜저급으로 맞추느라고...
1,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,원래는 쌍용차였는데\n음주운전 삼진 아웃되서 차를 매매하고\n추후에 이차를중고로 구...
2,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,전체적으로좋아요 승차감도 좋아요\n전체적으로좋아요 승차감도 좋아요\n전체적으로좋아요...
3,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,전반적으로 만족하지만 연비는 좀 떨어지는 편임\n빼고 잔고장도없고 전반적으로 만족함
4,2012 알페온,https://search.naver.com/search.naver?where=ne...,한국지엠,2012,준대형 세단,정숙성이우수함이 느껴져서 좋습니다..코너링과 급재동우수한것같습니다..~~
...,...,...,...,...,...,...
185813,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,기아,1990,소형 세단,처음 산 내 차.\n25년된 올드카지만 생각보다 멀쩡하고 연비도 괜찮다.\n웨건이라...
185814,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,기아,1990,소형 세단,1.3 sohc 개솔린 엔진은 특유의 저회전 토크가 강해 800kg 남짓의 차체를 ...
185815,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,현대,1990,준중형 세단,쓸만한 차\n가성비 좋음\n연비좋음\n. .\...
185816,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,현대,1990,준중형 세단,부품구입이 아직은 어렵지 않어 운행하는데 지장없고 엔진빼고 올 교체(올도색&amp;...


In [25]:
df_total = pd.concat([df_total_1, df_total_2, df_total_3, df_total_4], ignore_index=True)
df_total

,second_car_list,second_url_list,제조사,출시년도,외형,comments
0,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,여자들이 타기 좋은 차에요 !\n트렁크 공간은 넓으나 뒷자석은 좁구요\n차체 자체 ...
1,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,저는 차알못이고 신랑이 타는데\n연비는 진짜 미쳤구요\n이전에 미니 타다가 변경해서...
2,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,가격이 저렴하고 무난한 디자인. 기본 옵션들도 부족함이 별로 없다 생각합니다. 다만...
3,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,다 좋음 연비좋고 다 좋은데 서비스센터 구리고 엔진노킹 바퀴 소음 디스크문제 해결안...
4,2023 폭스바겐 제타,https://search.naver.com/search.naver?where=ne...,폭스바겐,2023,준중형 세단,시내주행은 연비 그닥이고 고속도로 정속주행시 연비 20키로 디자인은 개인적으로 깔끔...
...,...,...,...,...,...,...
1164901,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,기아,1990,소형 세단,처음 산 내 차.\n25년된 올드카지만 생각보다 멀쩡하고 연비도 괜찮다.\n웨건이라...
1164902,1990 프라이드 베타,https://search.naver.com/search.naver?where=ne...,기아,1990,소형 세단,1.3 sohc 개솔린 엔진은 특유의 저회전 토크가 강해 800kg 남짓의 차체를 ...
1164903,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,현대,1990,준중형 세단,쓸만한 차\n가성비 좋음\n연비좋음\n. .\...
1164904,1990 엘란트라,https://search.naver.com/search.naver?where=ne...,현대,1990,준중형 세단,부품구입이 아직은 어렵지 않어 운행하는데 지장없고 엔진빼고 올 교체(올도색&amp;...


In [26]:
df_total.to_csv('/content/drive/MyDrive/프로젝트_1/df_total.csv', encoding='utf-8-sig')

# 불용어/감성사전 

In [ ]:
# 코랩에서 Mecab 설치 
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ge

In [ ]:
import numpy as np
import pandas as pd
import json
import re
from tqdm.notebook import tqdm
import seaborn as sns

from konlpy.tag import Mecab
from collections import Counter

In [ ]:
df_review = pd.read_table('/content/drive/MyDrive/프로젝트_1/total_comment_set.txt', header=None, names=['reviews'])
df_review

,reviews
0,쉐보레 차를 원래 좋아해요. 묵직한 느낌이 안전한 느낌이 들어서 세번째 쉐보레만 타...
1,숫자상 출력 대비 동급 타사 모델보다 초반 출력은 월등히 차이가 납니다. 가격대비 ...
2,ㄱㅊ
3,ㅗㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅗㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅓㅓㅗㅓㅎ
4,유동성있게 어디나 타고다니기도 좋고 좁은 공간 주차 하기도 좋아요^^아주 만족해요
...,...
455753,연비는 15-16키로정도 (고속도로5 시내5 기준)
455754,현재까지 고장이력이 없다 현 13만KM
455755,평타 이상은 치는 차입니다.
455756,차 좋아요!! 디자인도 좋구.. 연비는 안좋은거 같아요!!!!!!!!!


In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455758 entries, 0 to 455757
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   reviews  455758 non-null  object
dtypes: object(1)
memory usage: 3.5+ MB


In [ ]:
mecab = Mecab()

In [ ]:
# 불용어 사전 - 리스트 

f = open('/content/drive/MyDrive/프로젝트_1/stopwords.txt')
stopwords = f.read().split()
stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과',
 '같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에',
 '이르다',
 '결국',
 '결론을',
 '낼',
 '수',
 '있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가',
 '있다',
 '관계없이',
 '관련이',
 '있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런',
 '까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도',
 '불구하고',
 '그렇게',
 '함으로써',
 '그렇지',
 '그렇지',
 '않다면',
 '그렇지',
 '않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에',
 '따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지',
 '않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지',
 '미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지',
 '않다',
 '놀라다',
 '누가',
 '알겠는가',
 '누구',
 '다른',
 '다른',
 '방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시',
 '말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지'

In [ ]:
words = []
for review in tqdm(df_review['reviews']):
    word = mecab.morphs(review)
    word = [review for review in word if len(review) > 1]
    words.extend(word)

print(words)

count = Counter(words)

display(count)

  0%|          | 0/455758 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Counter({'쉐보레': 1327,
         '원래': 444,
         '해요': 7590,
         '묵직': 2398,
         '느낌': 9922,
         '안전': 5595,
         '어서': 12190,
         '어요': 19031,
         '숫자': 34,
         '출력': 4650,
         '대비': 17860,
         '동급': 4886,
         '타사': 861,
         '모델': 5893,
         '보다': 22911,
         '초반': 1837,
         '월등히': 198,
         '차이': 3380,
         '납니다': 865,
         '가격': 32230,
         '옵션': 13394,
         '우월': 87,
         '전반': 12281,
         '으로': 56942,
         '만족감': 904,
         '습니다': 79801,
         'ㅗㅓ': 13,
         'ㅓㅓ': 81,
         'ㅗㅎ': 11,
         'ㅎㅎㅎ': 4327,
         'ㅓㅓㅗㅓㅎ': 1,
         '유동': 9,
         '어디': 1221,
         '다니': 7716,
         '공간': 14430,
         '주차': 5997,
         '아요': 46932,
         '^^': 4017,
         '아주': 17011,
         '만족': 67589,
         '편안': 6421,
         '나갑니다': 1907,
         '충분히': 1792,
         '합니다': 58779,
         '하지만': 5739,
         '스피커': 470,
         '아쉽': 8830,
      

In [ ]:
type(count) # 

collections.Counter

In [ ]:
sort_count = sorted(count.items(), key = lambda x: x[1], reverse=True)
sort_count

[('연비', 105595),
 ('습니다', 79801),
 ('만족', 67589),
 ('디자인', 61167),
 ('합니다', 58779),
 ('으로', 56942),
 ('주행', 50667),
 ('지만', 50593),
 ('입니다', 48630),
 ('아요', 46932),
 ('차량', 36767),
 ('성능', 34059),
 ('가격', 32230),
 ('네요', 31231),
 ('에서', 30758),
 ('생각', 29668),
 ('고장', 28680),
 ('최고', 23779),
 ('가성', 23601),
 ('보다', 22911),
 ('승차감', 21924),
 ('너무', 21531),
 ('는데', 21050),
 ('정도', 19989),
 ('어요', 19031),
 ('괜찮', 18871),
 ('대비', 17860),
 ('실내', 17755),
 ('아주', 17011),
 ('많이', 15384),
 ('없이', 14701),
 ('공간', 14430),
 ('조금', 14189),
 ('엔진', 13736),
 ('옵션', 13394),
 ('매우', 13299),
 ('까지', 13272),
 ('운전', 13218),
 ('다만', 12660),
 ('전반', 12281),
 ('..', 12214),
 ('어서', 12190),
 ('정말', 11582),
 ('무난', 11398),
 ('나오', 11238),
 ('단점', 10933),
 ('이나', 10858),
 ('시내', 10644),
 ('소음', 10536),
 ('해서', 10379),
 ('10', 10340),
 ('불편', 10241),
 ('구매', 10139),
 ('성비', 9976),
 ('느낌', 9922),
 ('이상', 9639),
 ('다고', 9601),
 ('안정', 9589),
 ('고속', 9420),
 ('라고', 9303),
 ('아직', 9280),
 ('아서', 9182),
 ('면서', 907

In [ ]:
len(sort_count)

45956